# Import the Python package

In [ ]:
import numpy as np
import Data_analysis as da
import matplotlib.pyplot as plt
import pandas as pd
import os
from scipy.stats import linregress

# Enter the path of the data you want to process

In [ ]:
PATH = "C:/Users/Computer_F6454-2/Documents/GitHub/EthierLab/stimulation.csv"
PATH_saving = ""

#### Information of the rat

In [ ]:
events_header = "events" # Enter the channel for the events
isobestic_header = "Isobestic" # Enter the channel for the isobestic channel
grabDA_header = "GrabDA" # Enter the channel for the grabDA channel

# Convert CSV to Numpy array
* Always enter the variable name in the order: time, events, isobestic and grabDA.
* If you want to have access to AnalogIn and/or AnalogOut, just enter AnalogIn=True and/or AnalogOut=True as an argument of the search_file function
* If an error say that a channel does not exist, change the parameter channel_events to another channel written in the printed message. (i.e. DIO04 -> channel_events=4)

In [ ]:
dictio = da.csv_to_np_arrays(PATH)

print(dictio)

keys_list = []
# Add keys from the dictionary to the list
for key in dictio.keys():
    keys_list.append(key)
try:
    time =  dictio[keys_list[0]]
    events = dictio[events_header]
    isobestic = dictio[isobestic_header]
    grabDA = dictio[grabDA_header]
except KeyError:
    print("The channel name you entered you entered does not exist")

# Cut the arrays so they all have the same length and find other important informations

In [ ]:
# Make array with the same length
events, isobestic, time, grabDA = da.check_time_value(events, isobestic, time, grabDA)

# Indentify the signals
stim, stim_name = grabDA, 'grabDA'
other, other_name = isobestic, "isobestic"

# Find the frequency
freq = da.find_frequency(time)

# Signal processing

#### Apply a Low-Pass filter

In [ ]:
### Enter True in is_artifact if you want to remove artifacts ###
is_artifact = False

# Plot the raw signals
plt.figure(figsize=(30, 6))
plt.plot(time, stim, label=stim_name, color='red')
plt.plot(time, other, label=other_name, color='purple')
plt.plot(time, events / 20, color='black', label="Events")
plt.title("Raw signals")
plt.xlabel("Time [s]")
plt.ylabel("Signal intensity")
plt.legend(loc="upper right")

plt.show()


# Denoise the two signals
denoised_stim = da.denoising(stim, 10, 5, freq)
denoised_other = da.denoising(other, 10, 5, freq)

# Remove the artifact from the two signals
if is_artifact:
    da.artifact_removal(denoised_stim, time)
    da.artifact_removal(denoised_other, time)

# Plot the signal without artifact and denoised
plt.figure(figsize=(30, 6))
plt.plot(time, denoised_stim, label=f"Denoised {stim_name}", color='red')
plt.plot(time, denoised_other, label=f"Denoised {other_name}", color='purple')
plt.plot(time, events / 20, color='black', label='Events')

title = "Denoised signal"
if is_artifact:
    title = title + " without artifact"

plt.title(title)
plt.xlabel("Time [s]")
plt.ylabel("Signal intensity")
plt.legend(loc="upper right")

plt.show()

#### Detrend the signal

In [ ]:
from scipy.sparse import csc_matrix, eye, diags
from scipy.sparse.linalg import spsolve

lambd = 7e7 # Adjust lambda to get the best fit
porder = 1
itermax = 50

fit_stim=da.airPLS(denoised_stim,lambda_=lambd,porder=porder,itermax=itermax)
fit_other=da.airPLS(denoised_other,lambda_=lambd,porder=porder,itermax=itermax)

detrend_stim = denoised_stim - fit_stim
detrend_other = denoised_other - fit_other

# Plot the detrended signals
plt.figure(figsize=(30, 6))

plt.plot(time, denoised_stim, color='red', label="GrabDA")
plt.plot(time, fit_stim, label="grabDA fit")

plt.plot(time, denoised_other, color='purple', label="Isobestic")
plt.plot(time, fit_other, label="Isobestic fit")

plt.plot(time, events / 200, color='black', label="Events")

plt.xlabel("Time [s]")
plt.ylabel("Signal intensity")
plt.title("Signals with their trend from the airPLS algorithm")
plt.legend(loc="upper right")

plt.show()

plt.figure(figsize=(30, 6))

plt.plot(time, detrend_other, label=f"Detrended {other_name}", color='purple')

plt.plot(time, detrend_stim, label=f"Detrended {stim_name}", color='red')

plt.plot(time, events / 200, color='black', label="Events")

plt.title("Detrended signals")
plt.xlabel("Time [s]")
plt.ylabel("Signal intensity")
plt.legend(loc="upper right")

plt.show()

#### We normalize the signal with the z-score method

In [ ]:
# Normalize the signals with the z score method
z_score_stim = da.normalize(detrend_stim)
z_score_other = da.normalize(detrend_other)


# Plot the normalized signals
plt.figure(figsize=(30, 6))
plt.plot(time, z_score_stim, label=f"Normalized {stim_name}", color='red')
plt.plot(time, z_score_other, label=f"Normalized {other_name}", color='purple')
plt.plot(time, events, color='black', label="Events")
plt.title("Normalized by z scoring signals")
plt.xlabel("Time [s]")
plt.ylabel("Signal intensity")
plt.legend(loc="upper right")

#### Motion correction

In [ ]:
# Reprensentation of the grabDA and isobestic correclation by scattering the two signals
plt.scatter(z_score_stim[::5], z_score_other[::5], alpha=0.01, color='black')

# Make a "mask" to ignore the NaN values
mask = ~np.isnan(z_score_stim) & ~np.isnan(z_score_other)

# Make a linear regression with the scipy.stats module
slope, intercept, r_value, p_value, std_err = linregress(x=z_score_stim[mask], y=z_score_other[mask])


# Plot the line
x = np.array(plt.xlim())
plt.plot(x, intercept + x*slope, color='red')
plt.title(f'{stim_name} - {other_name} correlation')
plt.xlabel(stim_name)
plt.ylabel(other_name)

plt.show()


# Do the motion correction
stim_corrected = da.motion_correction(z_score_stim, z_score_other, z_score_stim)
other_corrected = da.motion_correction(z_score_stim, z_score_other, z_score_other)


# Plot the signal with motion correction
plt.figure(figsize=(30, 6))
plt.plot(time, stim_corrected, label=f"Motion corrected {stim_name}", color='red')
plt.plot(time, other_corrected, label=f"Motion corrected {other_name}", color='purple')
plt.plot(time, events, color='black', label="Events")
plt.title("Motion corrected signals")
plt.xlabel("Time [s]")
plt.ylabel("Signal intensity")
plt.legend(loc="upper right")

plt.show()

#### dF/F

In [ ]:
# Make the dF/F on the signal so we can do the PSTH with it
df_f_signal = stim_corrected - other_corrected


# Plot the resulting signal with the events
plt.figure(figsize=(30, 6))
plt.plot(time, df_f_signal, label=f"dF/F {stim_name}", color='black')
plt.plot(time, events, color='red', label="events")
plt.title("dF/F signals")
plt.xlabel("Time [s]")
plt.ylabel("Signal intensity")
plt.legend(loc="upper right")

plt.show()

#### Then we save the new signal

In [ ]:
file_data = {
    'column1': time, # Create an array of time the same as the graphic above
    'column2': events, 
    'column3': df_f_signal
}


filename = PATH[PATH.rfind("/")+1:-4] + "_processed_signal.csv"
da.create_Processed_sig_CSV(PATH, PATH_saving+ filename, file_data)
